In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
from transformers import AutoTokenizer, AutoModel,TFBertForSequenceClassification
import numpy as np
from transformers import TFAutoModel
from tensorflow.keras import layers
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel

# Load the BERT tokenizer and the pretrained Arabic base BERT model
bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
# Load the BERT model and the pretrained Arabic base BERT model
model = TFAutoModel.from_pretrained("asafaya/bert-base-arabic")
#get the liste of sentences
"""
When feeding text data into our model, there are a few things to 
be aware of. First, we must use tokenizer.encode_plus(...) 
to convert our text into input IDs and attention mask tensors
"""
import tensorflow as tf

def tokenize_bert(data):
    sentences = data['setences'].values
    seq_len = 512
    num_samples = len(sentences)
    #lists of ids
    input_ids = np.zeros((num_samples, seq_len))
    #get the lists of masks
    attention_masks = np.zeros((num_samples, seq_len))
    for i,sent in enumerate(sentences):
        #tokenize words in the sentence
        #return_tensors: return the resultat as tensor
        #pad_to_max_length :pads the sentences to max length
        bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True
                                            ,max_length =seq_len,pad_to_max_length = True,
                                             truncation=True
                                            ,return_attention_mask = True,
                                             return_tensors='tf')  # Return pytorch tensors.
        #add encoded 
        input_ids[i, :] = bert_inp['input_ids']
        #applying attention masks
        attention_masks[i, :] = bert_inp['attention_mask']
    return input_ids,attention_masks


def get_model():
    # Load the BERT tokenizer and the pretrained Arabic base BERT model
    bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
    # Load the BERT model and the pretrained Arabic base BERT model
    model1 = TFAutoModel.from_pretrained("asafaya/bert-base-arabic")
    model1.trainable = False
    max_len = 512
    #'input_ids', 'attention_mask'
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
    embeddings = model1.bert(
            input_ids, attention_mask=attention_mask
        )[1]
    x = tf.keras.layers.Dense(519, activation='relu')(embeddings)
    #output layer
    y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)
    model1 = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=y)
    
    return model1

In [ ]:
neg = "../input/twitter/Negative"
pos = "../input/twitter/Positive"
positiveList=[]
negativeList=[]

negative = os.listdir(neg) 
postive = os.listdir(pos)

for file in negative:
    with open('../input/twitter/Negative/'+file,encoding="UTF-8") as f:
        try:
            negativeList.append(f.read())
        except:
            pass
for file in postive:
    with open(pos+"/"+file,encoding="UTF-8") as f:
        try:
            positiveList.append(f.read())
        except:
            pass
print(len(positiveList))
print(len(negativeList))
data={"setences":positiveList,"positive":[1 for i in range(len(positiveList))]}
df = pd.DataFrame.from_dict(data)
data1={"setences":negativeList,"positive":[0 for i in range(len(negativeList))]}
df1 =pd.DataFrame.from_dict(data1)
df = df.append(df1)
from sklearn.utils import shuffle
RANDOM_SEED = 14
#shuffle the data randomly
data = shuffle(df, random_state=RANDOM_SEED)

In [ ]:
from sklearn.model_selection import train_test_split
split = int((80*len(data))/100)
#train 
train = data[:split]
test = data[split:]
#get the inputs tokens and masks
train_inp,train_mask = tokenize_bert(train)
test_inp,test_mask = tokenize_bert(test)

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
#categorical_crossentropy
from transformers import TFAutoModel
from tensorflow.keras import layers
import tensorflow as tf
# Load the BERT model and the pretrained Arabic base BERT model
model = get_model()
model.summary()
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit([train_inp,train_mask],train['positive'],epochs=16,verbose=2)

In [ ]:
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=["accuracy"])
test_lost ,test_acc_model = model.evaluate(
[test_inp,test_mask],test['positive']
)
print("The accuracy of the model model is:",(test_acc_model*100))

In [ ]:
name="BERT sentiment analysis"
name = name.replace(' ','-')
model.save(f'{name}.h5')

# Rumors analysis

In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize
from transformers import TFAutoModel
from transformers import AutoTokenizer
import string
import tensorflow as tf
#text feature exctraction
def sentences_count(sentences):
    tokens = sent_tokenize(sentences)
    return len(tokens)
def word_count(sentences):
    tokens = word_tokenize(sentences)
    punctuations = string.punctuation
    return sum(1 for token in tokens if token not in punctuations)
def distanctWords(sentences):
    tokens = set(word_tokenize(sentences))
    punctuations = string.punctuation
    return sum(1 for token in tokens if token not in punctuations)
def avg_word_count(sentences):
    try:
        sentences = sent_tokenize(sentences)
        result = sum(word_count(sentence) for sentence in sentences)/len(sentences)
        return result
    except:
        return 0
def questionMarks(sentences):
    return sum(1 for token in sentences if (token =="؟") )
def exclamationMarks(sentences):
    return sum(1 for token in sentences if (token =="!") )
def negation(sentences):
    negationList=['لن','لم','لا','ليس','لستم','ليست','ليسوا']
    sentences = sent_tokenize(sentences)
    return sum(1 for token in sentences if token in negationList)
def calculValues(sentences):
    sentCont=sentences_count(sentences)
    wordCont=word_count(sentences)
    disWordCont=distanctWords(sentences)
    agvWordCont=avg_word_count(sentences)
    quesMark = questionMarks(sentences)
    exclamationMark = exclamationMarks(sentences)
    negate = negation(sentences)
    # Load the BERT model and the pretrained Arabic base BERT model
    bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
    bert_inp=bert_tokenizer.encode_plus(sentences,add_special_tokens = True
                                            ,max_length =512,pad_to_max_length = True,
                                             truncation=True
                                            ,return_attention_mask = True,
                                             return_tensors='tf') 
    sent = tf.keras.models.load_model(f'./BERT-sentiment-analysis.h5')
    sentiment = sent.predict([bert_inp['input_ids'],bert_inp['attention_mask']])
    sentiment=list(sentiment[0])
    sentiment_calc = sentiment[0] if sentiment[0] > sentiment[1] else sentiment[1]
    return [sentCont,wordCont,disWordCont,agvWordCont,quesMark,exclamationMark,negate,sentiment.index(sentiment_calc)]



In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize
from transformers import TFAutoModel
from transformers import AutoTokenizer
import string
import tensorflow as tf
#text feature exctraction
def sentences_count(sentences):
    tokens = sent_tokenize(sentences)
    return len(tokens)
def word_count(sentences):
    tokens = word_tokenize(sentences)
    punctuations = string.punctuation
    return sum(1 for token in tokens if token not in punctuations)
def distanctWords(sentences):
    tokens = set(word_tokenize(sentences))
    punctuations = string.punctuation
    return sum(1 for token in tokens if token not in punctuations)
def avg_word_count(sentences):
    try:
        sentences = sent_tokenize(sentences)
        result = sum(word_count(sentence) for sentence in sentences)/len(sentences)
        return result
    except:
        return 0
def questionMarks(sentences):
    return sum(1 for token in sentences if (token =="؟") )
def exclamationMarks(sentences):
    return sum(1 for token in sentences if (token =="!") )
def negation(sentences):
    negationList=['لن','لم','لا','ليس','لستم','ليست','ليسوا']
    sentences = sent_tokenize(sentences)
    return sum(1 for token in sentences if token in negationList)
def calculValues(sentences):
    sentCont=sentences_count(sentences)
    wordCont=word_count(sentences)
    disWordCont=distanctWords(sentences)
    agvWordCont=avg_word_count(sentences)
    quesMark = questionMarks(sentences)
    exclamationMark = exclamationMarks(sentences)
    negate = negation(sentences)
    # Load the BERT model and the pretrained Arabic base BERT model
    bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
    bert_inp=bert_tokenizer.encode_plus(sentences,add_special_tokens = True
                                            ,max_length =512,pad_to_max_length = True,
                                             truncation=True
                                            ,return_attention_mask = True,
                                             return_tensors='tf') 
    sent = tf.keras.models.load_model(f'./BERT-sentiment-analysis.h5')
    sentiment = sent.predict([bert_inp['input_ids'],bert_inp['attention_mask']])
    sentiment=list(sentiment[0])
    sentiment_calc = sentiment[0] if sentiment[0] > sentiment[1] else sentiment[1]
    return [sentCont,wordCont,disWordCont,agvWordCont,quesMark,exclamationMark,negate,sentiment.index(sentiment_calc)]


In [ ]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")
#import data from csv file
data = pd.read_csv('../input/covidremors/rumours_dataset.csv')
data.head()

In [ ]:
#drop unnecery rows
data = data.drop(["post url","post page","post date","likes","shares number","comments"], axis=1)
data.head()

In [ ]:
data['post text']=data['post text'].fillna('بدون نص')
data[data['post text']=='بدون نص']

# Encode output

In [ ]:
data['type'] = data['type'].apply(lambda x: 1 if x=='VRAI' else 0)
data.head()

In [ ]:
error=0
infos = []
cols = ['sentencesCount', 'wordCount','distanctWords','avg_word_count','questionMarks','exclamationMarks','negation','sentiment']
for i, row in data.iterrows():
    print(i)
    try:
        features = calculValues(row['post text'])
        infos.append(features)
    except Exception as e:
        print(e)
df = pd.DataFrame(infos,columns =cols)
data1 = pd.concat([data, df], axis=1, join="inner")


In [ ]:
data1.head()

In [ ]:
# Load the BERT model and the pretrained Arabic base BERT model
model1 = TFAutoModel.from_pretrained("asafaya/bert-base-arabic")

model1.trainable = False  #freeze the weight

input_ids = layers.Input(shape=(512,), dtype=tf.int32, name='input_ids')
attention_mask = layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask')
features = layers.Input(shape=(8,), dtype=tf.float32, name='features1')
embeddings = model1.bert(
            input_ids, attention_mask=attention_mask
        )[1]
x = tf.keras.layers.Concatenate()([embeddings, features])
x = tf.keras.layers.Dense(599, activation='relu')(x)
#output layer
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)
model1 = tf.keras.Model(inputs=[input_ids, attention_mask,features], outputs=y)
model1.summary()
# get the input and the output shapes of the BERT model
from keras.models import Model
BERT_model = Model(model1.input, model1.output)

In [ ]:
split = int((80*len(data))/100)
from sklearn.utils import shuffle

data1 = shuffle(data1)
#train 
train = data1.iloc[:split,:]
test = data1.iloc[split:,:]
images_train = images=create_dataset(train)
images_test = images=create_dataset(test)
train = train.drop(['post image'],axis=1)

In [ ]:
train

In [ ]:
test

# images processing

## calcul the local binary pattern

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
def get_pixel(img, center, x, y):
      
    new_value = 0
      
    try:
        # If local neighbourhood pixel 
        # value is greater than or equal
        # to center pixel values then 
        # set it to 1
        if img[x][y] >= center:
            new_value = 1      
    except:
        pass
      
    return new_value
   
# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):
   
    center = img[x][y]
   
    val_ar = []
      
    # top_left
    val_ar.append(get_pixel(img, center, x-1, y-1))
      
    # top
    val_ar.append(get_pixel(img, center, x-1, y))
      
    # top_right
    val_ar.append(get_pixel(img, center, x-1, y + 1))
      
    # right
    val_ar.append(get_pixel(img, center, x, y + 1))
      
    # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y + 1))
      
    # bottom
    val_ar.append(get_pixel(img, center, x + 1, y))
      
    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y-1))
      
    # left
    val_ar.append(get_pixel(img, center, x, y-1))
       
    # Now, we need to convert binary
    # values to decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]    
    return val

def LBP(img_bgr):
    height, width, _ = img_bgr.shape
    # We need to convert RGB image 
    # into gray one because gray 
    # image has one channel only.
    img_gray = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2GRAY)

    # Create a numpy array as 
    # the same height and width 
    # of RGB image
    img_lbp = np.zeros((height, width),
                       np.uint8)

    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    return img_lbp

def divide_image_by_block(image):
    blocks=[]
    img_x=image.shape[0]
    img_y=image.shape[1]
    for x in range(img_x):
        for y in range(img_y):
            blocks.append(image[x:x+16][y:y+16])
    return blocks

## define the DDT

In [ ]:
from scipy.fftpack import dct
def ddt_for_blocks(image):
    dct_vectors=[]
    for x in range(16):
        dct_vectors.append(dct(dct(image[x]).T).T)
    return np.array(dct_vectors)
def ddt_matrx_std(image):
    image = LBP(image)
    image = divide_image_by_block(image)
    image = ddt_for_blocks(image)
    ddt_std=[]
    for x in range(16):
        ddt_std.append(image[x].std())
    return ddt_std

In [ ]:
import cv2
import numpy as np
import os
#load images from the disk
def create_dataset(data):
    #importing images from the disk
    path="../input/covidremors/"
    img_data_array=[]
    class_name=[]
    #new height and width of images
    IMG_HEIGHT, IMG_WIDTH = 224,224
    #iterate throw csv file to get the name of images
    for index,file in data[['post image']].iterrows():
        #get image path
        img_path = data.loc[index,['post image']].values[0]
        image_path= os.path.join(path,img_path)
        #read image via opencv
        image= cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        #resize the image into IMG_HEIGHT*IMG_WIDTH*3
        image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH ),interpolation = cv2.INTER_AREA)
        #convert image to numpy array
        image=np.array(image)
        #applying DDT on images
        ddt_std = ddt_matrx_std(image)
        #convert image to float32
        image = image.astype('float32')
        #normalise the image via deviding all the values by 255
        image /= 255
        img_data_array.append(image)
        #append image to class_name
        class_name.append(data.loc[index,['type']].values[0])
    return np.array(img_data_array)



In [ ]:
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
#load pretrained ResNet152V2
ResNet152V2 = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#freewe the weights 
ResNet152V2.trainable = False
# add custom output layers for ResNet
#get the output of the model
x = ResNet152V2.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=117, activation='relu')(x)
x = Dense(units=1080, activation='relu')(x)
x = Dropout(0.7)(x)
ResNet_output  = Dense(units=2, activation='softmax')(x)
ResNet_model = Model(ResNet152V2.input, ResNet_output)

In [ ]:
from keras.layers.merge import concatenate
#concatenate the outputs of the 2 models
merge_layer = concatenate([ResNet_output, BERT_model.output], name='Concatenate')
out = Dense(1048, activation='relu', name='output_layer')(merge_layer)
out = Dropout(0.5)(out)
final_model_output = Dense(2, activation='sigmoid',name="final_layer")(out)
merged_model = Model([ResNet152V2.input, BERT_model.input], outputs=final_model_output)
merged_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.9999999, epsilon=1e-06, decay=0.1),
                     loss = "sparse_categorical_crossentropy" ,
                     metrics=["accuracy"])


In [ ]:
from transformers import AutoTokenizer, AutoModel,TFBertForSequenceClassification
import numpy as np
from transformers import TFAutoModel
from tensorflow.keras import layers
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel
# Load the BERT tokenizer and the pretrained Arabic base BERT model
bert_tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
# Load the BERT model and the pretrained Arabic base BERT model
model = TFAutoModel.from_pretrained("asafaya/bert-base-arabic")
#get the liste of sentences
"""
When feeding text data into our model, there are a few things to 
be aware of. First, we must use tokenizer.encode_plus(...) 
to convert our text into input IDs and attention mask tensors
"""
import tensorflow as tf

def tokenize_bert(data):
    sentences = data['post text'].values
    seq_len = 512
    num_samples = len(sentences)
    #lists of ids
    input_ids = np.zeros((num_samples, seq_len))
    #get the lists of masks
    attention_masks = np.zeros((num_samples, seq_len))
    for i,sent in enumerate(sentences):
        #tokenize words in the sentence
        #return_tensors: return the resultat as tensor
        #pad_to_max_length :pads the sentences to max length
        bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True
                                            ,max_length =seq_len,pad_to_max_length = True,
                                             truncation=True
                                            ,return_attention_mask = True,
                                             return_tensors='tf')  # Return pytorch tensors.
        #add encoded 
        input_ids[i, :] = bert_inp['input_ids']
        #applying attention masks
        attention_masks[i, :] = bert_inp['attention_mask']
    return input_ids,attention_masks

In [ ]:
split = int((80*len(data))/100)
from sklearn.utils import shuffle

data1 = shuffle(data1)
#train 
train = data1.iloc[:split,:]
test = data1.iloc[split:,:]
images_train =create_dataset(train)
images_test = images=create_dataset(test)
train = train.drop(['post image'],axis=1)

In [ ]:
train

In [ ]:
train_inp,train_mask = tokenize_bert(train)
test_inp,test_mask = tokenize_bert(test)

In [ ]:
train_inp,train_mask = tokenize_bert(train)
test_inp,test_mask= tokenize_bert(test)
labels_train = train[['type']]
labels_test = test[['type']]

merged_model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=["accuracy"])
train_x = [np.array(images_train),[train_inp,train_mask, train.iloc[:,2:]  ]]
merged_model.fit(train_x,labels_train, epochs=30, steps_per_epoch=50)

In [ ]:
test_lost ,test_acc_model = merged_model.evaluate(
[np.array(images_test), [test_inp,test_mask, test.iloc[:,3:].values ]  ],labels_test
)
print("The accuracy of the model model is:",(test_acc_model*100))

In [ ]:
def get_metrics(y_pred,y_list):
    FP,FN,TP,TN=0,0,0,0
    for i in range(len(y_list)):
        if y_pred[i]==y_list[i]:
            if y_pred[i]==1:
                TP+=1
            else:
                TN+=1
        else:
            if y_pred[i]==0:
                FP+=1 
            else: 
                FN+=1
    return FP,FN,TP,TN
def accuracy(y_pred,y_list):
    FP,FN,TP,TN=get_metrics(y_pred,y_list)
    return (TP+TN)/len(y_list)
def recall(y_pred,y_list):
    FP,FN,TP,TN=get_metrics(y_pred,y_list)
    return TP/(TP+FN)
def precision(y_pred,y_list):
    FP,FN,TP,TN=get_metrics(y_pred,y_list)
    return TP/(TP+FP)
def f1_score(y_pred,y_list):
    prec,rec=recall(y_pred,y_list),precision(y_pred,y_list)
    return (2*prec*rec)/(prec+rec)


y_list = list( labels_test['type'] )

predicted = merged_model.predict(  [np.array(images_test), [test_inp,test_mask, test.iloc[:,3:].values ]  ] )
y_pred = list(predicted)
y_pred = [ list(i).index(max(i[0],i[1])) for i in predicted ]
print(accuracy(y_pred,y_list))
print(recall(y_pred,y_list))
print(precision(y_pred,y_list))
print(f1_score(y_pred,y_list))